In [1]:
!pip install -U pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg pydicom python-gdcm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 55.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 72.5 MB/s eta 0:00:00:00:010:01


In [ ]:
!pip install pylibjpeg --no-index --find-links=file:///kaggle/input/read-dicom-set
!pip install pylibjpeg-openjpeg --no-index --find-links=file:///kaggle/input/read-dicom-set
!pip install pylibjpeg-libjpeg --no-index --find-links=file:///kaggle/input/read-dicom-set
!pip install pydicom --no-index --find-links=file:///kaggle/input/read-dicom-set
!pip install python-gdcm --no-index --find-links=file:///kaggle/input/read-dicom-set
!pip install dicomsdl --no-index --find-links=file:///kaggle/input/read-dicom-set


Looking in links: file:///kaggle/input/read-dicom-set
Looking in links: file:///kaggle/input/read-dicom-set
Looking in links: file:///kaggle/input/read-dicom-set
Looking in links: file:///kaggle/input/read-dicom-set
Looking in links: file:///kaggle/input/read-dicom-set


In [ ]:
!pip install natsort


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pydicom
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pylibjpeg
from libjpeg import decode
import pydicom as dicom
from pydicom import dcmread
from pydicom.data import get_testdata_file
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from natsort import natsorted
import cv2
import math

In [ ]:
data_path='/kaggle/input/iaaa-mri-challenge'

In [ ]:
len_of_slices = []
for i in os.listdir(f'{data_path}/data'):
    lenn=len(os.listdir(f'{data_path}/data/{i}'))
    if lenn==20:
        print(i)
    len_of_slices.append(lenn)


In [ ]:
plt.figure(figsize=(10, 6))
counts, bins, patches = plt.hist(len_of_slices, bins=range(min(len_of_slices), max(len_of_slices) + 2), alpha=0.6, color='blue', edgecolor='black', align='left')

# Calculate and display frequency values
bin_centers = 0.5 * (bins[:-1] + bins[1:])
for count, x in zip(counts, bins[:-1]):
    plt.text(x, count, f'{int(count)}', ha='center', va='bottom')

plt.title('Histogram of Length of Slices with Frequency Values')
plt.xlabel('Length of Slices')
plt.ylabel('Frequency')
plt.show()

In [ ]:
img=dicom.dcmread('/kaggle/input/iaaa-mri-challenge/data/1.3.46.670589.11.10042.5.0.1412.2024020313391873234/1.3.46.670589.11.10042.5.0.1412.2024020313401406243.dcm')

In [ ]:
img.pixel_array.shape

In [ ]:
slice_number_mr_tag = (0x2001, 0x100a)

In [ ]:
img[(0x2001, 0x100a)].value

In [ ]:
total_height=[]
total_width=[]
total_aspect=[]
for i in os.listdir(f'{data_path}/data'):
    for j in os.listdir(f'{data_path}/data/{i}')[:1]:
        img=dicom.dcmread(f'{data_path}/data/{i}/{j}')
        height,width=img.pixel_array.shape
        aspect_ratio=height/width
    total_height.append(height)
    total_width.append(width)
    total_aspect.append(aspect_ratio)
        

In [ ]:
plt.figure(figsize=(10, 6))
counts, bins, patches = plt.hist(total_height, bins=range(min(total_height), max(total_height) + 2), alpha=0.6, color='blue', edgecolor='black', align='left')

# Calculate and display frequency values
bin_centers = 0.5 * (bins[:-1] + bins[1:])

plt.title('Histogram of Length of Slices with Frequency Values')
plt.xlabel('Length of Slices')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
counts, bins, patches = plt.hist(total_width, bins=range(min(total_width), max(total_width) + 2), alpha=0.6, color='blue', edgecolor='black', align='left')

# Calculate and display frequency values
bin_centers = 0.5 * (bins[:-1] + bins[1:])

plt.title('Histogram of Length of Slices with Frequency Values')
plt.xlabel('Length of Slices')
plt.ylabel('Frequency')
plt.show()

 all aspect ratios are 1

In [ ]:
def prepare_dataset(paths):
    dataset=[]

    for i,path in enumerate(paths):
        img=dicom.dcmread(path,force=True)
        pip=img.PhotometricInterpretation

        data=img.pixel_array
        data=data-np.min(data)
        if np.max(data) != 0:
            data=data/np.max(data)
        data=(data*255).astype(np.uint8)  
        data=cv2.cvtColor(data,cv2.COLOR_GRAY2RGB)
        data=cv2.cvtColor(data,cv2.COLOR_BGR2GRAY)
        #data=cv2.resize(data,dsize=(224,224)) # to plot x,y, do not resize the images
        if pip=='MONOCHROME2':
            data=255-data
        ps=path.split('/')[-4:]
        newfile=ps[-3]+'_'+ps[-2]+'_'+ps[-1][0:-4].zfill(4)+'.png'
        cv2.imwrite(newfile,data)
        if data.max()>0:
            dataset+=[(newfile,data)]

    return dataset

In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')

def create_animation(ims):    
    fig=plt.figure(figsize=(4,4))
    im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
    text = plt.text(0.05, 0.05, f'Slide {1}', transform=fig.transFigure, fontsize=14, color='blue')
    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
        text.set_text(f'Slide {i+1}')        
        return [im] 
    plt.axis('off')
    plt.gray()
    plt.close()    

    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000//10)

8 slice len:

1.3.46.670589.11.10042.5.0.3364.2024011115503556469

1.3.46.670589.11.10042.5.0.7620.2023122617465726464

1.3.46.670589.11.10042.5.0.7756.2023121515490121921

all the 8 slices are without any problem

13 slice len:

1.3.46.670589.11.10042.5.0.7756.2023121417345950250

from slices 8 to the end should move to first

1.3.46.670589.11.10042.5.0.7756.2023121417373448299

8th slice is the 2nd 9th is 3rd and etc

14 slice len:

1.3.46.670589.11.10042.5.0.3364.2024011123452729203

from 1 to 7 and from 8 to 14 should be moved together

1.3.46.670589.11.10042.5.0.3364.2024011123440200172

from 8 to 14 should move to first

In [ ]:
def sort_dicom_files(paths):
    sorted_files=[np.nan]*24
    for i in range(len(paths)):
        dicom=pydicom.dcmread(paths[i])
        slice_number_mr_tag=(0x2001, 0x100a)
        slice_number_mr_value=int(dicom[slice_number_mr_tag].value)
        sorted_files[(slice_number_mr_value-1)]=paths[i]
    filtered_paths = [path for path in sorted_files if not isinstance(path, float)]

    return filtered_paths

In [ ]:
paths=[]
path1='/kaggle/input/iaaa-mri-challenge/data/1.3.46.670589.11.10042.5.0.6580.2024020906123971463'
for i in os.listdir(path1):
    paths.append(f'{path1}/{i}')
paths=sort_dicom_files(paths)
dataset1=prepare_dataset(paths)
images1 = [data[1] for data in dataset1]

In [ ]:
create_animation(images1)